In [1]:
import warnings
warnings.simplefilter("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
%matplotlib inline
import gensim
import tensorflow as tf
import keras.backend as K
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.initializers import Constant
from keras.layers import Layer
from keras.layers import (Input, Embedding, SpatialDropout1D, Bidirectional,
                          GlobalAveragePooling1D, GlobalMaxPooling1D,
                          LSTM, CuDNNLSTM, GRU, CuDNNGRU, concatenate, Dropout, Dense, Activation,
                          Lambda, Flatten, RepeatVector, Permute, Multiply, Conv1D)
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
NUM_WORDS = 100000
MAXLEN = 400
EMBEDDING_DIM = 300

In [3]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data):
        self.X_val, self.y_val = validation_data
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        score = roc_auc_score(self.y_val, y_pred)
        print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch + 1, score))

In [4]:
train = pd.read_csv('data/train_preprocessed.csv')
test = pd.read_csv('data/test_preprocessed.csv')
train['comment_text'].fillna("", inplace=True)
test['comment_text'].fillna("", inplace=True)
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
X_train_raw, y_train = train['comment_text'].str.lower(), train[classes].values
X_test_raw = test['comment_text'].str.lower()
tk = Tokenizer(num_words=NUM_WORDS)
tk.fit_on_texts(X_train_raw)
X_train = tk.texts_to_sequences(X_train_raw)
X_test = tk.texts_to_sequences(X_test_raw)
X_train = pad_sequences(X_train, maxlen=MAXLEN)
X_test = pad_sequences(X_test, maxlen=MAXLEN)
nb_words = min(NUM_WORDS, len(tk.word_index) + 1)

In [5]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def get_embedding(pretrained_word_vectors):
    if pretrained_word_vectors == "google":
        EMBEDDING_PATH = "../embeddings/GoogleNews-vectors-negative300.bin"
        embedding_index = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_PATH, binary=True)
    if pretrained_word_vectors == "glove":
        EMBEDDING_PATH = "../embeddings/glove.840B.300d.txt"
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_PATH))
    elif pretrained_word_vectors == "fasttext":
        EMBEDDING_PATH = "../embeddings/crawl-300d-2M.vec"
        embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(EMBEDDING_PATH))
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in tk.word_index.items():
        if i >= NUM_WORDS:
            continue
        if pretrained_word_vectors == "google":
            try:
                embedding_vector = embedding_index.get_vector(word)
            except Exception:
                continue
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [6]:
def attention_weighted(x):
    x1 = Dense(1)(x)
    x1 = Flatten()(x1)
    x1 = Activation("softmax")(x1)
    x1 = RepeatVector(128)(x1)
    x1 = Permute((2, 1))(x1)
    x1 = Multiply()([x, x1])
    return Lambda(lambda x: K.sum(x, axis=1))(x1)

In [7]:
def build_model(embedding_matrix):
    inputs = Input(shape=(MAXLEN,))
    x = Embedding(nb_words, EMBEDDING_DIM,
                  embeddings_initializer=Constant(embedding_matrix),
                  trainable=False)(inputs)
    x = SpatialDropout1D(0.25)(x)
    filter_nums = 128
    conv_0 = Conv1D(filter_nums, 1, padding="valid", activation="relu")(x)
    conv_1 = Conv1D(filter_nums, 2, padding="valid", activation="relu")(x)
    conv_2 = Conv1D(filter_nums, 3, padding="valid", activation="relu")(x)
    conv_3 = Conv1D(filter_nums, 4, padding="valid", activation="relu")(x)
    maxpool_0 = GlobalMaxPooling1D()(conv_0)
    attn_0 = attention_weighted(conv_0)
    avg_0 = GlobalAveragePooling1D()(conv_0)
    maxpool_1 = GlobalMaxPooling1D()(conv_1)
    attn_1 = attention_weighted(conv_1)
    avg_1 = GlobalAveragePooling1D()(conv_1)
    maxpool_2 = GlobalMaxPooling1D()(conv_2)
    attn_2 = attention_weighted(conv_2)
    avg_2 = GlobalAveragePooling1D()(conv_2)
    maxpool_3 = GlobalMaxPooling1D()(conv_3)
    attn_3 = attention_weighted(conv_3)
    avg_3 = GlobalAveragePooling1D()(conv_3)
    hidden = concatenate([maxpool_0, maxpool_1, maxpool_2, maxpool_3,
                          attn_0, attn_1, attn_2, attn_3,
                          avg_0, avg_1, avg_2, avg_3])
    x = Dropout(0.5)(hidden)
    x = Dense(128, activation="relu")(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer=Adam(decay=1e-6), metrics=['accuracy'])
    return model

In [8]:
embedding_matrix = get_embedding("fasttext")

In [9]:
n_splits = 10
kf = KFold(n_splits=10, shuffle=True, random_state=0)
train_pred = np.zeros((X_train.shape[0], 6))
pred = np.zeros((X_test.shape[0], 6))
for train_index, test_index in kf.split(X_train, y_train):
    model = build_model(embedding_matrix=embedding_matrix)
    roc_eval = RocAucEvaluation(validation_data=(X_train[test_index], y_train[test_index]))
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    model.fit(X_train[train_index], y_train[train_index], batch_size=128, epochs=50,
              validation_data=(X_train[test_index], y_train[test_index]),
              callbacks = [roc_eval, early_stopping])
    train_pred[test_index] = model.predict(X_train[test_index], batch_size=1024, verbose=1)
    pred += model.predict(X_test, batch_size=1024, verbose=1)
print("oof auc :", roc_auc_score(y_train, train_pred))
pred /= n_splits
submission = pd.read_csv("data/sample_submission.csv")
submission[classes] = pred
submission.to_csv("submission/v6_CNN_fasttext_preprocess_submission.csv.gz", compression="gzip", index=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 143613 samples, validate on 15958 samples
Epoch 1/50
143613/143613 [==============================] - 41s 283us/step - loss: 0.0583 - acc: 0.9791 - val_loss: 0.0424 - val_acc: 0.9833

 ROC-AUC - epoch: 1 - score: 0.984621
Epoch 2/50
143613/143613 [==============================] - 38s 264us/step - loss: 0.0444 - acc: 0.9825 - val_loss: 0.0430 - val_acc: 0.9833

 ROC-AUC - epoch: 2 - score: 0.989343
Epoch 3/50
143613/143613 [==============================] - 38s 262us/step - loss: 0.0417 - acc: 0.9834 - val_loss: 0.0404 - val_acc: 0.9837

 ROC-AUC - epoch: 3 - score: 0.989968
Epoch 4/50
143613/143613 [==============================] - 38s 267us/step - loss: 0.0401 - acc:

143614/143614 [==============================] - 39s 273us/step - loss: 0.0317 - acc: 0.9869 - val_loss: 0.0386 - val_acc: 0.9844

 ROC-AUC - epoch: 14 - score: 0.990455
153164/153164 [==============================] - 11s 72us/step
Train on 143614 samples, validate on 15957 samples
Epoch 1/50
143614/143614 [==============================] - 41s 285us/step - loss: 0.0584 - acc: 0.9789 - val_loss: 0.0435 - val_acc: 0.9826

 ROC-AUC - epoch: 1 - score: 0.985239
Epoch 2/50
143614/143614 [==============================] - 38s 267us/step - loss: 0.0438 - acc: 0.9828 - val_loss: 0.0417 - val_acc: 0.9836

 ROC-AUC - epoch: 2 - score: 0.988408
Epoch 3/50
143614/143614 [==============================] - 39s 273us/step - loss: 0.0418 - acc: 0.9833 - val_loss: 0.0395 - val_acc: 0.9841

 ROC-AUC - epoch: 3 - score: 0.989221
Epoch 4/50
143614/143614 [==============================] - 39s 271us/step - loss: 0.0400 - acc: 0.9839 - val_loss: 0.0401 - val_acc: 0.9841

 ROC-AUC - epoch: 4 - score: 0.989


 ROC-AUC - epoch: 8 - score: 0.990238
Epoch 9/50
143614/143614 [==============================] - 40s 278us/step - loss: 0.0350 - acc: 0.9857 - val_loss: 0.0394 - val_acc: 0.9836

 ROC-AUC - epoch: 9 - score: 0.990799
Epoch 10/50
143614/143614 [==============================] - 40s 279us/step - loss: 0.0342 - acc: 0.9859 - val_loss: 0.0392 - val_acc: 0.9836

 ROC-AUC - epoch: 10 - score: 0.990534
Epoch 11/50
143614/143614 [==============================] - 40s 279us/step - loss: 0.0333 - acc: 0.9862 - val_loss: 0.0406 - val_acc: 0.9827

 ROC-AUC - epoch: 11 - score: 0.990302
Epoch 12/50
143614/143614 [==============================] - 40s 278us/step - loss: 0.0329 - acc: 0.9865 - val_loss: 0.0401 - val_acc: 0.9834

 ROC-AUC - epoch: 12 - score: 0.990180
153164/153164 [==============================] - 11s 72us/step
Train on 143614 samples, validate on 15957 samples
Epoch 1/50
143614/143614 [==============================] - 42s 294us/step - loss: 0.0575 - acc: 0.9795 - val_loss: 0.044